# Measurements for single qubit systems

<!-- The **Measurements** quantum kata is a series of exercises designed
to get you familiar with the concept of measurements and with programming in Q#.
It covers the following topics:
* single-qubit measurements,
* discriminating orthogonal and nonorthogonal states.
 -->
<!-- Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS). -->

<!-- The tasks are given in approximate order of increasing difficulty; harder ones are marked with asterisks. -->

This tutorial introduces you to measurements done on single-qubit systems. The concept of a measurement is a central part of quantum mechanics, as well as quantum algorithms. Single-qubit measurements, as their name implies are measurements on single qubits. The outcomes of a measurement in quantum mechanics are probabilistic, and in general, change the state of the qubit depending on the outcome of the measurement. 

We recommend to go through the [tutorial that introduces single qubit gates](../SingleQubitGates/SingleQubitGates.ipynb) before starting this one.

This tutorial covers the following topics for single qubit systems:

* Computational basis measurements
* Pauli basis measurements
* Measurements in arbitrary orthogonal bases



$\newcommand{\ket}[1]{\left|#1\right>}$
$\newcommand{\bra}[1]{\left<#1\right|}$
# The Basics
In this section, we will discuss the basics of qubit measurements, starting with measurements in the computational basis.

The state $\ket{\psi}$ of a single qubit can always be expressed in [Dirac notation](../Qubit/Qubit.ipynb#Dirac-Notation) as 
\begin{align*}
    \ket{\psi} = \alpha \ket{0} + \beta \ket{1},
\end{align*}
where $\alpha$ and $\beta$ are complex numbers, and the state is assumed to be normalized, so that $|\alpha|^2 + |\beta|^2 = 1$. Then, the outcome of a measurement done in the computational basis can either be $0$, or $1$. The probabilities of the measurement outcomes being $0$ and $1$ are $|\alpha|^2$ and $|\beta|^2$ respectively. Additionally, the state of the qubit is modified by the measurement. If the outcome of the measurement is $0$, then the post-measurement state of the qubit is $\ket{0}$ (and similarly for $1$). In quantum mechanics, this is referred to as the [collapse of a wavefunction](https://en.wikipedia.org/wiki/Wave_function_collapse).

Computational basis measurement outcomes and their probabilites are summarized in the table below:
<table style="border:1px solid">
    <col width=150>
    <col width=150>
    <col width=150>
    <tr>
        <th style="text-align:center; border:1px solid">Measurement Outcome</th>
        <th style="text-align:center; border:1px solid">Probability of outcome</th>
        <th style="text-align:center; border:1px solid"> Wavefunction after measurement</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$0$</td>
        <td style="text-align:center; border:1px solid">$|\alpha|^2$</td>
        <td style="text-align:center; border:1px solid">$\ket 0$</td>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$1$</td>
        <td style="text-align:center; border:1px solid">$|\beta|^2$</td>
        <td style="text-align:center; border:1px solid">$\ket 1$</td>
    </tr>
    
</table>

> Unlike quantum gates which are unitary and reversible operations, in general, measurements in quantum mechanics are neither unitary nor reversible. Since outcomes of a measurement are probabilistic, any two isolated qubits which are initially prepared in identical superposition states are in general not guaranteed to have the same measurement outcomes after each qubit has been measured separately. As we will see below, measurements are modelled by projection operators instead of unitary operators.
>
> Additionally, the assumption of the wavefunction being **normalized** is important for the applicablility of the statements above. This is because the probability outcomes must sum up to $1$. If the wavefunction is not normalized, it is important to first normalize it first, in order to obtain the measurement probabilities.


### <span style="color:blue">Exercise 1</span>: Obtain the probability outcomes for a specific state

Suppose that a qubit is known to be in the following state:
$$\ket \psi = 0.6 \ket 0 + 0.8 \ket 1 \equiv \begin{bmatrix} 0.6 \\ 0.8\end{bmatrix}.$$

If the qubit is measured in the computational basis, what are the outcome probabilities?

*Can't come up with a solution? See the explained solution in the [Single Qubit System Measurements Tutorial Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-1:-Obtain-the-probability-outcomes-for-a-specific-state).*


### <span style="color:blue">Demo: Measurement using the M operation</span>


In this demo, we first prepare a qubit in the state corresponding to Exercise 1, and measure the qubit in the computational (or Pauli Z) basis. In Q#, single qubit measurements in the computational (i.e. the Pauli Z) basis can be implemented by using the [M operation](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.m). 


In [ ]:
// Run this cell using Ctrl+Enter (⌘+Enter on Mac)
// Run the next cell to see the output

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;


operation M_Measurement_Demo () : Unit {
    let divider = "--------------------------------------------------------------------------------------------------";
    using (q = Qubit()) {
        
        // This will put the qubit into an the superposition state |𝜓❭
        // with |𝜓❭ = 0.6 |0❭ + 0.8 |1❭
        Ry(2.0 * ArcTan2(0.8, 0.6), q);

        Message("Qubit in state |𝜓❭:");
        DumpMachine();
        Message(divider);
        
        // Let us measure the qubit in the PauliZ basis
        mutable outcome = 1;
        if (M(q) == One){
            set outcome = 1;
        } else {
        set outcome = 0;
        }

        Message($"The measurement outcome is {outcome}.");
        Message("Post-measurement state of the qubit:");
        DumpMachine();
        Message(divider);
        
        // This returns the qubit into state |0❭
        Reset(q);
    }
}


In [ ]:
%simulate M_Measurement_Demo

> If you run the cell above multiple times, you will notice that whenever the measurement outcome is $1$, the post-measurement state of the qubit also corresponds to $\ket 1$, and similarly for $0$. This is in line with our expectation that the wavefunction 'collapses' after a measurement to the corresponding state.

### <span style="color:blue">Demo: Measurement statistics</span>

The following cell contains code demonstrating that the theoretical and experimental values of the probability outcomes indeed match with each other. To that end, we prepare the state $\ket \psi = 0.6 \ket 0 + 0.8 \ket 1$ and measure it in the computational basis $100$ times. At the end, we expect $0$ to be measured approximately $36$ times, and $1$ to be measured approximately $64$ times. Note that since measurements are probabilistic, we do not expect the results to match exactly with these values.

In [ ]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

operation M_statistics_Demo () : Unit {
    
    // Set countZero and countOne to 0.
    mutable countZero = 0;
    let numRuns = 100; // measure 100 times
    using (q = Qubit()) {
    
        for (i in 1 .. numRuns) {
        
            // This will put the qubit into an the superposition state |𝜓❭
            // with |𝜓❭ = 0.6 |0❭ + 0.8 |1❭
            Ry(2.0 * ArcTan2(0.8, 0.6), q);
            
            
            // Measure in the computational basis, and update the counts according to the outcomes
            
            if (M(q) == Zero) {
                set countZero += 1;
            } 
            // Reset the qubit for use in the next iteration
            Reset(q);
        }
    }
    let countOne = numRuns - countZero;
    
    Message($"Simulated probability of measuring 0 is 0.{countZero}.");
    Message($"Theoretical probabilty of measuring 0 is 0.36");
    
    
    Message($"Simulated probability of measuring 1 is 0.{countOne}.");
    Message($"Theoretical probabilty of measuring 0 is 0.64");
}



In [ ]:
%simulate M_statistics_Demo

Measurements can be used to distinguish between orthogonal states. We start with an exercise for distinguishing between the computational basis states, and discuss the general case of arbitrary basis measurements later in the tutorial.

### <span style="color:blue">Exercise 2</span>: Distinguish $|0\rangle$ and $|1\rangle$

**Input:** A qubit is known to be in either the $|0\rangle$ or the $|1\rangle$ state.

**Output:**  `true` if the qubit was in the $|0\rangle$ state, or `false` if it was in the $|1\rangle$ state. The state of the qubit at the end of the operation does not matter.

In [ ]:
%kata T2_IsQubitZero

operation IsQubitZero (q : Qubit) : Bool {
    // The operation M will measure a qubit in the Z basis (|0⟩ and |1⟩ basis)
    // and return Zero if the observed state was |0⟩ or One if the state was |1⟩.
    // To answer the question, you need to perform the measurement and check whether the result
    // equals One - either directly or using library function IsResultOne.
    //
    // Type the following: return M(q) == Zero;
    // Then run the cell using Ctrl+Enter (⌘+Enter on macOS).

    // ...
}

*Can't come up with a solution? See the explained solution in the [Single Qubit System Measurements Tutorial Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-2:-Distinguish-$|0\rangle$-and-$|1\rangle$).*

## Measurements in other Pauli bases
So far, we have discussed measurements done in the computational basis, i.e. the $\{ \ket 0, \ket 1\}$ basis. It is also possible to implement measurements in other orthogonal bases, such as the [Pauli-X basis](../SingleQubitGates/SingleQubitGates.ipynb#Pauli-Gates), which consists of the two vectors $\ket + = \frac{\ket 0 +\ket 1}{\sqrt{2}}$, and $\ket - = \frac{\ket 0 - \ket 1}{\sqrt{2}}$. Q# has an inbuilt operation called [Measure](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.measure) for measurements in the Pauli bases. The `Measure` operation can be used for measuring multiple qubits simultaneously in a multi-qubit system; however, here we only consider measurements for single qubit systems.

The eigenvalues of a Pauli matrix are $\pm 1$, with one eigenvector corresponding to each eigenvalue. For any chosen Pauli basis, the `Measure` operation returns `Zero` if the measurement outcome corresponds to the eigenvalue $+1$, and returns `One` if the measurement outcome corresponds to the eigenvalue $-1$. As in the case of the Pauli-Z measurements, the wavefunction of the qubit collapses to the corresponding state after the measurement is executed. 


### <span style="color:blue">Exercise 3</span>: Distinguish $|+\rangle$ and $|-\rangle$ using the `Measure` operation

**Input**: A qubit which is known to be in either the $\ket +$ state, or the $\ket -$ state.

**Output**: `true` if the qubit is in the $\ket -$ state, or `false` if it was in the $\ket +$ state. Obtain the output by using the `Measure` operation.


In [ ]:
%kata T3_IsQubitMinus

operation IsQubitMinus (q : Qubit) : Bool {
    // The operation Measure measures a qubit in the X, Y or Z basis,
    // and returns Zero if the observed state corresponds to an eigenvalue of $+1$,
    // and returns One if it corresponds to an eigenvalue of $-1$.
    // ...
}

*Can't come up with a solution? See the explained solution in the [Single Qubit System Measurements Tutorial Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-3:-Distinguish-$|+\rangle$-and-$|-\rangle$-using-the-Measure-operation).*

## Measurements in arbitrary orthogonal bases
It is possible to measure a qubit in orthogonal bases other than the Pauli bases. Suppose one wants to measure a qubit in an orthonormal basis $\ket b_0$ and $\ket b_1$. Let the state of the qubit be represented by the normalized vector $\ket \psi$. Then, one can always express the state in terms of the basis vectors $\ket{b_0}$ and $\ket{b_1}$, i.e. there exist complex numbers $c_0, c_1$, such that 
$$
\ket \psi = c_0 \ket {b_0} + c_1 \ket {b_1}.
$$
The rule for obtaining the probabilities of measurement outcomes are similar to those for the computation basis. Specifically, for a measurement in a $\{ b_0, b_1\}$ basis, we have
- Outcome $b_0$ with probability $|c_0|^2$, and the post-measurement state of the qubit being $\ket {b_0}$;
- Outcome $b_1$ with probability $|c_1|^2$, and the post-measurement state of the qubit being $\ket {b_1}$.

This can be summarized in the following table:
<table style="border:1px solid">
    <col width=150>
    <col width=150>
    <col width=150>
    <tr>
        <th style="text-align:center; border:1px solid">Measurement Outcome</th>
        <th style="text-align:center; border:1px solid">Probability of outcome</th>
        <th style="text-align:center; border:1px solid"> Wavefunction after measurement</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$b_0$</td>
        <td style="text-align:center; border:1px solid">$|c_0|^2$</td>
        <td style="text-align:center; border:1px solid">$\ket{b_0}$</td>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$b_1$</td>
        <td style="text-align:center; border:1px solid">$|c_1|^2$</td>
        <td style="text-align:center; border:1px solid">$\ket{b_1}$</td>
    </tr>
    
</table>

As before, the assumption of $\ket \psi$ being normalized is important, since it guarantees that the two probabilities add to one.


> As you may recall, a [global phase](../Qubit/Qubit.ipynb#Relative-and-Global-Phase) is said to be hidden or unobservalble. This is also consistent with the fact that global phases have no consequences on quantum measurements. For example, consider two isolated qubits which are in (normalized) states $\ket \psi$ and $e^{i\theta}\ket \psi$. If both are measured in an orthogonal basis $\{ \ket{b_0},\ket{b_1}\}$, the probabilities of measuring $b_0$ or $b_1$ are identical in both cases, since $|\bra{b_i}\ket{\psi}|^2 = |\bra{b_i}e^{i\theta}\ket{\psi}|^2  $. Similarly, for either qubit, if $b_i$ is the measurement outcome, the post-measurement state of the qubit is $\ket{b_i}$ for both qubits. Hence, the measurements are independent of the gloabl phase $\theta$.

### Measurements as projection operations
 Measurements done in orthogonal bases are modelled by orthogonal projection operators. An orthogonal projection operator (i.e. a matrix) $P$ satisfies the following property:
$$
P^2 = P^\dagger = P.
$$
(As usual, the $\dagger$ symbol denotes conjugate transposition.) Using the [ket-bra representation](../SingleQubitGates/SingleQubitGates.ipynb#Ket-bra-Representation), one can represent a projection matrix in the Dirac notation.
For example, one may construct a projector onto the $\ket{0}$ subspace as:
$$
P = \ket 0 \bra 0 \equiv \begin{bmatrix} 1 & 0 \\ 0 & 0\end{bmatrix}.
$$

A measurement in an orthogonal basis $\{ \ket{b_0}, \ket{b_1}\}$ is described by a pair of projectors $\ket{b_0}\bra{b_0}$, and $\ket{b_1}\bra{b_1}$. Let us denote these projectors by $P_0$ and $P_1$ respectively. Since $\ket{b_0}$ and $\ket{b_1}$ are mutually orthogonal, their projectors are also mutually orthogonal, i.e. $P_0 P_1 = P_1 P_0 = 0$. The rules for measurements in this basis can then be summarized as follows: 
- Measuring a qubit which is in a state $\ket \psi$ is done by picking one of these projection operators at random.
- Projection $P_0$ is chosen with a probability of $|P_0 \ket{\psi}|^2$, while the projector $P_1$ is chosen with a probability of $|P_1\ket{\psi}|^2$.
- If projector $P_0$ is chosen, the post-measurement state of the qubit is given by
$$
\frac{P_0 \ket\psi}{|P_0 \ket{\psi}|},
$$
and similarly for $P_1$.

Although this formalism might appear to be different than from the previous section, it is in fact equivalent to it. This can be seen by noting that with $\ket \psi = c_0 \ket{b_0} + c_1 \ket{b_1}$, we have 
\begin{align}
P_0 \ket \psi &= c_0 \ket{b_0}, \\
\text{so that } | P_0\ket \psi|  &= c_0,
\end{align}
and similarly, 
\begin{align}
P_1 \ket \psi &= c_1 \ket{b_1}, \\
\text{so that } |P_1\ket \psi|  &= c_1.
\end{align}
Thus, as before the probability of measuring $b_0$ is $|P_0\ket\psi|^2 = |c_0|^2$, and the probability of measuring $b_1$ is $|P_1\ket\psi|^2 = |c_1|^2$. Similarly, one can verify that the post-measurement outcomes are also $\ket{b_0}$ and $\ket{b_1}$ respectively (upto unphysical global phases).

Although the projecor formalism for single qubit systems may seem superfluous, it's importance will become clear while studying mesaurements for multi-qubit systems.


### Arbitrary Basis Measurement Implementation using gates and computational basis measurements
In a previous section, we discussed measurements in Pauli bases using the `Measure` operation. We will now show that using just unitary rotation matrices and computation basis measurements (i.e. using the `M` operation), it is always possible to measure a qubit not only in the Pauli bases, but in any orthogonal basis. 

Consider a state $ \ket \psi = c_0 \ket {b_0} + c_1 \ket {b_1} $ which we would like to measure in an orthonormal basis $\{ \ket{b_0}, \ket{b_1}\}$. First, we construct a unitary matrix $U$, using
$$
U = \ket{0} \bra{b_0} + \ket{1} \bra{b_1}.
$$

The Hermitian conjugate of this unitary is the operator 
$$
U^\dagger = \ket{b_0} \bra{0} + \ket{b_1} \bra{1}.
$$

One may verify that $U$ is indeed a unitary matrix, by checking that $U^\dagger U = U U^\dagger = 1_{2}$. First, we note that the action of these matrices on the two bases is the following:
\begin{align}
U\ket{b_0} &= \ket{0}; & U\ket{b_1} &= \ket{1}\\
\text{while }U^\dagger \ket{0} &= \ket{b_0}; & U^\dagger \ket 1 &= \ket{b_1}.
\end{align}

In order to implement a measurement in the $\{ \ket{b_0}, \ket{b_1}\}$ basis, we do the following:

1. Operate $U$ on $\ket \psi$. (The resultant state is $U\ket \psi = c_0 \ket 0 + c_1 \ket 1 $).
2. Measure the state $U\ket{\psi}$ in the computational basis. (The outcomes 0 and 1 occur with probabilities $|c_0|^2$ and $|c_1|^2$.)
3. Operate $U^\dagger$ on the measured state. (This transforms the states $\ket 0$ and $\ket 1$ to the states $\ket{b_0}$ and $\ket{b_1}$ respectively.

Thus, $b_0$ and $b_1$ are measured with probabilities $|c_0|^2$ and $|c_1|^2$ respectively, with the end state being $\ket{b_0}$ and $\ket{b_1}$ as desired. 

This procedure can be used to distinguish arbitrary orthogonal states using the `M` operation in conjunction with the appropriate quantum gates, as will become clear from the following exercises.

### <span style="color:blue">Exercise 4</span>: Obtain the probability outcomes for a measurement in a specified basis
<ol>
<li>Suppose that a qubit is known to be in the state $\ket{0}$. If the qubit is measured in the Pauli X basis, i.e. the $\{ \ket +, \ket -\}$ basis, what are the outcome probabilities?
</li>
<li>Suppose that a qubit is known to be in the state $0.6\ket{0} + 0.8 \ket{1}$. If the qubit is measured in the Pauli Y basis, i.e. the $\{ \ket i, \ket{-i}\}$ basis, what are the outcome probabilities?</li>
</ol> 

*Can't come up with a solution? See the explained solution in the [Single Qubit Measurement Tutorial Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-4:-Obtain-the-probability-outcomes-for-a-measurement-in-a-specified-basis).*

### <span style="color:blue">Exercise 5</span>: Distinguishing specific orthogonal states

**Inputs:** A qubit which is guaranteed to be in either the $\ket {\psi_+}$ or the $\ket{\psi_-} $ state, where $\ket {\psi_+} = 0.6\ket 0 + 0.8 \ket 1 $ and $\ket {\psi_-} = -0.8\ket 0 + 0.6 \ket 1$.

**Output:** `true` if the qubit was in the $\ket {\psi_+}$ state, or `false` if it was in the $\ket{\psi_-} $ state. The state of the qubit at the end of the operation does not matter.

<br/>
<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    A suitable <a href="../SingleQubitGates/SingleQubitGates.ipynb#Rotation-Gates">$R_y$ rotation</a> can be used to go from the computational basis $\{ \ket 0, \ket 1 \}$ to the $\{ \ket{\psi_+}, \ket{\psi_-} \}$ basis. 
     
</details>

In [ ]:
%kata T5_IsQubitPsiPlus

open Microsoft.Quantum.Math;

operation IsQubitPsiPlus (q : Qubit) : Bool{
    // ...
}

*Can't come up with a solution? See the explained solution in the [Single Qubit Measurement Tutorial Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-5:-Distinguishing-specific-orthogonal-states-in-Q#).*

### <span style="color:blue">Exercise 6</span>: $\ket A$ or $\ket B$?

**Inputs:** 

1. Angle $\alpha$, in radians, represented as a `Double`.
2. A qubit which is guaranteed to be in either the $|A\rangle$ or the $|B\rangle$ state, where $|A\rangle = \cos \alpha |0\rangle - i \sin \alpha |1\rangle$ and $|B\rangle = - i \sin \alpha |0\rangle + \cos \alpha |1\rangle$.

**Output:** `true` if the qubit was in the $|A\rangle$ state, or `false` if it was in the $|B\rangle$ state. The state of the qubit at the end of the operation does not matter.

<br/>
<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    An <a href="../SingleQubitGates/SingleQubitGates.ipynb#Rotation-Gates">$R_x$ rotation</a> can be used to go from the computational basis $\{ \ket 0, \ket 1 \}$ to the $\{ \ket{A}, \ket{B} \}$ basis. 
     
</details>

In [ ]:
%kata T6_IsQubitA

operation IsQubitA (alpha : Double, q : Qubit) : Bool {
    // ...
}

*Can't come up with a solution? See the explained solution in the [Single Qubit Measurement Tutorial Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-6:-$\ket-A$-or-$\ket-B$?).*

### <span style="color:blue">Exercise 7**</span>: Measurement in the $\ket A$, $\ket B$ basis

**Inputs:** 
1. A qubit in some unknown state.
1. Angle $\alpha$, in radians, represented as a `Double`.


**Output:** Implement a measurement in the $A/B$ basis. Return `Zero` if the measurement outcome is $A$, and `One` if the outcome is $B$. 
The state of the qubit after the measurement should be the same as the measured state.

As in the previous exercise, $|A\rangle = \cos \alpha |0\rangle - i \sin \alpha |1\rangle$ and $|B\rangle = - i \sin \alpha |0\rangle + \cos \alpha |1\rangle$.

<br/>
<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    An <a href="../SingleQubitGates/SingleQubitGates.ipynb#Rotation-Gates">$R_x$ rotation</a> can be used to go from the computational basis $\{ \ket 0, \ket 1 \}$ to the $\{ \ket{A}, \ket{B} \}$ basis. 
     
</details>

In [ ]:
%kata T7_MeasureInABBasis

operation MeasureInABBasis (alpha: Double, q : Qubit) : Result {
    // ...
}

*Can't come up with a solution? See the explained solution in the [Single Qubit Measurement Tutorial Workbook](./Workbook_SingleQubitSystemMeasurements.ipynb#Exercise-7**:-Measurement-in-the-$\ket-A$,-$\ket-B$-basis).*